## Vectorbt - Dual Bollinger Bands on single asset

Reference 1: https://greyhoundanalytics.com/blog/create-a-custom-indicator-in-vectorbt/ 

Reference 2: http://qubitquants.pro/strategydev/index.html

In [1]:
import vectorbt as vbt
import numpy

In [2]:
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
ohlcv = vbt.YFData.download("BTC-USD", missing_index="drop",
                            start="07-08-2023",
                            interval="5m").get(cols)
print(ohlcv)

                                   Open          High           Low  \
Datetime                                                              
2023-07-07 16:00:00+00:00  30410.642578  30412.316406  30403.056641   
2023-07-07 16:05:00+00:00  30391.021484  30396.808594  30385.951172   
2023-07-07 16:10:00+00:00  30395.443359  30434.644531  30395.443359   
2023-07-07 16:15:00+00:00  30414.837891  30414.837891  30387.546875   
2023-07-07 16:20:00+00:00  30387.925781  30387.925781  30379.187500   
...                                 ...           ...           ...   
2023-08-09 14:50:00+00:00  29895.902344  29895.914062  29821.681641   
2023-08-09 14:55:00+00:00  29791.964844  29794.466797  29765.621094   
2023-08-09 15:00:00+00:00  29760.111328  29760.111328  29734.552734   
2023-08-09 15:05:00+00:00  29736.867188  29757.417969  29736.867188   
2023-08-09 15:10:00+00:00  29746.001953  29746.001953  29746.001953   

                                  Close    Volume  
Datetime                

In [9]:
m15_data = ohlcv.resample("15T").last()
h4_data = ohlcv.resample("4h").last()


In [10]:
h4_BBANDS = vbt.BBANDS.run(h4_data.Close)
m15_BBANDS = vbt.BBANDS.run(m15_data.Close)
m15_RSI = vbt.RSI.run(m15_data.Close) # 15 * 1min = 15 min RSI

##### Trading Rules:

entry:
1. H4 "Low" price goes below its lower Bollinger band
2. 15m RSI goes below its lower Bollinger band

exit:
1. H4 "High" price breaks its upper Bollinger band
2. 15m RSI breaks above its upper Bollinger band

In [15]:
## The two variables `bb_upper_fract` and `bb_lower_fract` are simply some adjustment 
# parameters for the RSI bollinger bands and they are explained at the end of this article.

entries = h4_BBANDS.lower_above(h4_data.Low)\
               & m15_RSI.rsi_below(m15_BBANDS.upper)
exits = h4_BBANDS.upper_below(h4_data.High)\
                    & m15_RSI.rsi_above(m15_BBANDS.upper)

In [16]:
entries.sum()

12

In [17]:
exits.sum()

0

In [19]:
pf = vbt.Portfolio.from_signals(m15_data.Close,
                      entries,
                      exits,
                      direction="LongOnly",
                      init_cash=1000)

In [20]:
pf.stats()

Start                         2023-07-07 16:00:00+00:00
End                           2023-08-09 15:00:00+00:00
Period                                 32 days 23:15:00
Start Value                                      1000.0
End Value                                    989.386957
Total Return [%]                              -1.061304
Benchmark Return [%]                          -2.253083
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               5.367597
Max Drawdown Duration                  20 days 04:30:00
Total Trades                                          1
Total Closed Trades                                   0
Total Open Trades                                     1
Open Trade PnL                               -10.613043
Win Rate [%]                                        NaN
Best Trade [%]                                      NaN
Worst Trade [%]                                 

In [22]:
pf.plot().show()